In [2]:
#create virtual enviornment for this (run on cmd line)

#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#:~:text=Topic%20Modeling%20is%20a%20technique,in%20the%20Python's%20Gensim%20package.
#do above for topic extraction + consideration

In [3]:
# !pip install pandas
# !pip install nltk
# !pip install gensim
# !pip install spacy
# !pip install pyLDAvis
# !pip install matplotlib

In [4]:
#download prereqs
import nltk
nltk.download('stopwords')
# !python3 -m spacy download en
# !python3 -m spacy download en_core_web_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\logan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.6 MB/s eta 0:00:05
      --------------------------------------- 0.2/12.8 MB 2.1 MB/s eta 0:00:07
      --------------------------------------- 0.2/12.8 MB 2.2 MB/s eta 0:00:06
     - -------------------------------------- 0.3/12.8 MB 2.3 MB/s eta 0:00:06
     - -------------------------------------- 0.4/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.5/12.8 MB 1.8 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.8 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.8 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.8 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.8 MB 2.0 MB/s eta 0:00:07
     -- ------


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\logan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\logan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [6]:
#import prereq (per https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#:~:text=Topic%20Modeling%20is%20a%20technique,in%20the%20Python)
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
#define globals here for ease of replication
DATA_FILE = '../Result_college.csv'
BIGRAM_THRESHHOLD = 50
TRIGRAM_THRESHHOLD = 50

In [8]:
#read the dataset in
import pandas as pd

df = pd.read_csv(DATA_FILE)

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created,key_words
0,0,14.0,My Wife Attempted Suicide,489,vliro3,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,93,Wednesday afternoon my wife took a massive amo...,1.656291e+09,suicide
1,1,25.0,"My BF thinks suicidal people are all weak, cow...",487,kgaidi,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,136,This is really just a rant I have to get off m...,1.608394e+09,suicide
2,2,31.0,New wiki on how to avoid accidentally encourag...,1787,cz6nfd,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,248,We've been seeing a worrying increase in pro-s...,1.567526e+09,suicide
3,3,69.0,Prepare for Suicide?,3,17bzh65,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,15,"Hi everyone, sorry if this post is a little me...",1.697766e+09,suicide
4,4,101.0,Suicide?,1,179hh4q,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,0,Hey guys I don’t usually come on here to expla...,1.697492e+09,suicide


In [10]:
#clean the data
# data = df.values.tolist()

#tokenize using gensim (8)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


df["txt"] = df["title"] + " " + df["body"]
df["txt_raw"] = df["title"] + " " + df["body"]
for key_change in ["txt"]:
    print(df[key_change].to_list())

    _values = df[key_change].to_list()

    _values = [re.sub('\s+', ' ', sent) if sent and isinstance(sent, type("")) else "" for sent in _values]
    _values = [re.sub("\'", "", sent) if sent and isinstance(sent, type("")) else "" for sent in _values]
    _values = list(sent_to_words(df[key_change]))

    df[key_change] = _values

# df["all_txt"] = df["body"] + df["title"]
# df["title"] = df["title"].astype(str)
# df["body"] = df["body"].astype(str)
# df["all_txt"] = df["title"].str.cat(' ', df["body"])


['My Wife Attempted Suicide Wednesday afternoon my wife took a massive amount of medication. She texted me a photo of two piles of pills and said "Goodbye. I\'m sorry." I was at my parents apartment in the same building as ours, so I ran to our apartment, but she locked me out. I beat on the door over and over yelling for her to open up, and when she finally did, she just stumbled across the floor and fell on her bed, gagging and crying. I called 911 and held her head up and tried to keep her awake. She was vomiting everywhere. The paramedics carried her out of the apartment and down to a stretcher and I sped to the ER right behind them. They gave me a number to call in the ER to get updates, and all they kept saying was that she was in very critical condition and they\'re doing their best to stabilize her. She was vomiting so much that they couldn\'t pump her stomach and had to put her on a ventilator. Hours went by that felt like days, and the Dr finally called and said that they sta

In [11]:
df.head()

,Unnamed: 0.1,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created,key_words,txt,txt_raw
0,0,14.0,My Wife Attempted Suicide,489,vliro3,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,93,Wednesday afternoon my wife took a massive amo...,1.656291e+09,suicide,"[my, wife, attempted, suicide, wednesday, afte...",My Wife Attempted Suicide Wednesday afternoon ...
1,1,25.0,"My BF thinks suicidal people are all weak, cow...",487,kgaidi,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,136,This is really just a rant I have to get off m...,1.608394e+09,suicide,"[my, bf, thinks, suicidal, people, are, all, w...","My BF thinks suicidal people are all weak, cow..."
2,2,31.0,New wiki on how to avoid accidentally encourag...,1787,cz6nfd,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,248,We've been seeing a worrying increase in pro-s...,1.567526e+09,suicide,"[new, wiki, on, how, to, avoid, accidentally, ...",New wiki on how to avoid accidentally encourag...
3,3,69.0,Prepare for Suicide?,3,17bzh65,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,15,"Hi everyone, sorry if this post is a little me...",1.697766e+09,suicide,"[prepare, for, suicide, hi, everyone, sorry, i...","Prepare for Suicide? Hi everyone, sorry if thi..."
4,4,101.0,Suicide?,1,179hh4q,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,0,Hey guys I don’t usually come on here to expla...,1.697492e+09,suicide,"[suicide, hey, guys, don, usually, come, on, h...",Suicide? Hey guys I don’t usually come on here...


In [12]:
#get words for topic modelling - TODO segregrate this based on different topics maybe
data_words = df["txt"]
bigram = gensim.models.Phrases(data_words, min_count=5,threshold=BIGRAM_THRESHHOLD)
trigram = gensim.models.Phrases(bigram[data_words], threshold=TRIGRAM_THRESHHOLD)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [14]:
# !python3 -m spacy download en_core_web_sm
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [15]:
#(10) - remove stopwords, bigram + lemmatize functions 

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

# import en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# nlp = en_core_web_sm.load()

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['wife', 'afternoon', 'wife', 'take', 'massive', 'amount', 'medication', 'texte', 'photo', 'pile', 'pill', 'say', 'goodbye', 'sorry', 'parent', 'apartment', 'building', 'run', 'apartment', 'lock', 'beat', 'door', 'yell', 'open', 'finally', 'stumble', 'floor', 'fall', 'bed', 'gag', 'crying', 'call', 'held', 'head', 'try', 'keep', 'awake', 'vomiting', 'everywhere', 'paramedic', 'carry', 'apartment', 'stretcher', 'speed', 'right', 'behind', 'give', 'number', 'call', 'get', 'update', 'keep', 'say', 'critical', 'condition', 'well', 'stabilize', 'vomit', 'much', 'pump', 'stomach', 'put', 'ventilator', 'hour', 'go', 'feel', 'day', 'finally', 'call', 'say', 'stabilize', 'ventilator', 'amount', 'duloxetine', 'ingest', 'high', 'chance', 'brain', 'dead', 'brain', 'damage', 'dad', 'hold', 'feel', 'wind', 'leave', 'lung', 'dad', 'take', 'phone', 'talk', 'pick', 'kid', 'get', 'car', 'say', 'gon', 'pull', 'wait', 'drive', 'home', 'pass', 'couch', 'nurse', 'call', 'tell', 'get', 'ventilator', 'negati

In [17]:
#create dictioanary + corpus for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 3), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 4), (21, 2), (22, 1), (23, 1), (24, 6), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 2), (39, 2), (40, 4), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 3), (60, 3), (61, 3), (62, 1), (63, 2), (64, 1), (65, 1), (66, 7), (67, 1), (68, 10), (69, 1), (70, 2), (71, 1), (72, 1), (73, 4), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 7), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 5), (87, 4), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [18]:
#build topic model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
#13 - view topics in LDA
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"hrs_week" + 0.012*"student" + 0.012*"develop" + 0.011*"counselor" + '
  '0.011*"event" + 0.009*"biology" + 0.008*"article" + 0.008*"disorder" + '
  '0.007*"local" + 0.007*"release"'),
 (1,
  '0.025*"get" + 0.025*"college" + 0.022*"class" + 0.016*"go" + 0.015*"school" '
  '+ 0.014*"take" + 0.013*"work" + 0.013*"year" + 0.013*"know" + 0.012*"make"'),
 (2,
  '0.015*"feel" + 0.014*"people" + 0.014*"go" + 0.013*"get" + 0.012*"know" + '
  '0.010*"life" + 0.010*"even" + 0.010*"say" + 0.009*"friend" + 0.009*"time"'),
 (3,
  '0.018*"help" + 0.013*"talk" + 0.012*"suicide" + 0.009*"resource" + '
  '0.008*"support" + 0.007*"mental_health" + 0.006*"suicidal" + 0.005*"call" + '
  '0.005*"daily" + 0.005*"charity"'),
 (4,
  '0.024*"school" + 0.014*"essay" + 0.011*"also" + 0.011*"teacher" + '
  '0.011*"interview" + 0.010*"write" + 0.009*"good" + 0.008*"year" + '
  '0.008*"apply" + 0.007*"reject"')]


In [20]:
#score model
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.422312772941834

Coherence Score:  0.37953785146699753


In [ ]:
#visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [21]:
#TODO - upgrade to mallet (current gensim no longer supports - can decide if want to do)
lda_mallet = lda_model

In [22]:
#17 - chose optimal # topics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        # model = gensim.models.wrappers.LdaMallet(MALLET_PATH, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

START, LIMIT, STEP = 2,20,1
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=START, limit=LIMIT, step=STEP)

In [23]:

x = range(START, LIMIT, STEP)
best_idx = -1
best_score = coherence_values[-1]
idx = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    if (cv >= best_score): 
        best_idx = idx 
        best_score = cv
    
    idx += 1

Num Topics = 2  has Coherence Value of 0.4278
Num Topics = 3  has Coherence Value of 0.4137
Num Topics = 4  has Coherence Value of 0.3669
Num Topics = 5  has Coherence Value of 0.3795
Num Topics = 6  has Coherence Value of 0.4222
Num Topics = 7  has Coherence Value of 0.4192
Num Topics = 8  has Coherence Value of 0.3857
Num Topics = 9  has Coherence Value of 0.4453
Num Topics = 10  has Coherence Value of 0.451
Num Topics = 11  has Coherence Value of 0.4993
Num Topics = 12  has Coherence Value of 0.4801
Num Topics = 13  has Coherence Value of 0.4719
Num Topics = 14  has Coherence Value of 0.4241
Num Topics = 15  has Coherence Value of 0.3555
Num Topics = 16  has Coherence Value of 0.4376
Num Topics = 17  has Coherence Value of 0.4132
Num Topics = 18  has Coherence Value of 0.4173
Num Topics = 19  has Coherence Value of 0.4482


In [24]:
best_score

0.4992517211103591

In [25]:
#select the model and print the topics
#TODO - change to take idx of best one
optimal_model = model_list[-1]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.153*"bio" + 0.146*"file" + 0.074*"attendance" + 0.067*"master" + '
  '0.059*"researcher" + 0.009*"devastate" + 0.008*"thesis" + 0.006*"willpower" '
  '+ 0.005*"collapse" + 0.005*"envy"'),
 (1,
  '0.134*"school" + 0.051*"write" + 0.048*"apply" + 0.044*"research" + '
  '0.042*"student" + 0.040*"application" + 0.031*"list" + 0.031*"program" + '
  '0.028*"honor" + 0.026*"submit"'),
 (2,
  '0.234*"reject" + 0.098*"rejection" + 0.056*"disorder" + 0.042*"autism" + '
  '0.027*"believe" + 0.026*"office" + 0.024*"teen" + 0.020*"sound" + '
  '0.017*"professional" + 0.017*"appeal"'),
 (3,
  '0.005*"wildlife" + 0.005*"employer" + 0.004*"hearted" + 0.003*"being" + '
  '0.002*"ofcourse" + 0.001*"pessimistic" + 0.000*"homesickness" + '
  '0.000*"faker" + 0.000*"robotic" + 0.000*"google"'),
 (4,
  '0.474*"essay" + 0.126*"award" + 0.059*"defer" + 0.033*"predict" + '
  '0.031*"write" + 0.025*"relatively" + 0.023*"incredibly" + 0.023*"hr" + '
  '0.015*"mine" + 0.015*"semi"'),
 (5,
  '0.097*"furi

In [26]:
#visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.347694  0.137244       1        1  35.132095
14     0.377175  0.184048       2        1  30.887970
12     0.414816  0.125557       3        1  24.579793
1      0.279104 -0.340853       4        1   5.019363
2      0.073451 -0.183984       5        1   1.861100
8     -0.039304  0.056309       6        1   0.794861
4      0.082653 -0.254819       7        1   0.690876
6     -0.113550  0.006370       8        1   0.293253
17    -0.102354 -0.003443       9        1   0.203109
0     -0.116687  0.008997      10        1   0.141279
16    -0.137433  0.031147      11        1   0.081491
10    -0.136746  0.029838      12        1   0.069454
15    -0.137204  0.031252      13        1   0.065509
7     -0.135927  0.028270      14        1   0.058618
9     -0.113312  0.006509      15        1   0.058346
5     -0.137034  0.032665      16        1   0.037938
3     -0.135440  0.034753      17        1   0.018467
13    -0.134951  0.035070      18        1   0.003282
18    -0.134951  0.035070      19        1   0.003194, topic_info=        Term         Freq        Total Category  logprob  loglift
897   school  3407.000000  3407.000000  Default  30.0000  30.0000
1084  reject  1127.000000  1127.000000  Default  29.0000  29.0000
77      help  1342.000000  1342.000000  Default  28.0000  28.0000
59      feel  3503.000000  3503.000000  Default  27.0000  27.0000
2320   essay   850.000000   850.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
25       car     0.000739   184.719426  Topic19  -9.3251  -2.0782
26     carry     0.000739    18.278765  Topic19  -9.3251   0.2349
27    chance     0.000739   167.314640  Topic19  -9.3251  -1.9792
28     child     0.000739   131.880888  Topic19  -9.3251  -1.7413
29    clothe     0.000739    22.187385  Topic19  -9.3251   0.0411

[998 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1233      2  1.000488      absolutely
429       1  0.999561          accept
1757      8  0.996649  accomplishment
434       1  0.999265        activity
0         1  0.569954        actually
...     ...       ...             ...
203       1  0.530738            year
203       2  0.012140            year
203       3  0.457140            year
1936      2  0.999297             yet
2403      4  0.999921           youth

[604 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 15, 13, 2, 3, 9, 5, 7, 18, 1, 17, 11, 16, 8, 10, 6, 4, 14, 19])

In [27]:
#find dominant topic https://stackoverflow.com/questions/63777101/topic-wise-document-distribution-in-gensim-lda
import sys 

all_topics = optimal_model.print_topics()
docs_per_topic = [ [] for _ in all_topics]

for doc_id, doc_bow in enumerate(corpus):
    doc_topics = optimal_model.get_document_topics(doc_bow)
    for topic_id, score in doc_topics:
        docs_per_topic[topic_id].append((score, doc_id))

for doc_list in docs_per_topic:
    doc_list.sort()
    doc_list.reverse()




for topic_num in range(len(all_topics)):
    print("topic list is ", all_topics[topic_num])
    these_docs = docs_per_topic[topic_num]
    _input = "n" 
    idx = 0
    while (idx < len(these_docs) and _input == "n"):
        print("\n"*3)
        this_doc = these_docs[idx][1]

        # print(" ".join(data_lemmatized[this_doc]))
        print(idx)
        print(df["txt_raw"][this_doc])
        idx += 1

        #flush
        sys.stdout.flush()
        _input = input("n if next one of these")
        while (_input not in ["b","c","n"]): _input = input("n if next one of these")
    if (_input == "b"):
        break
    

topic list is  (0, '0.153*"bio" + 0.146*"file" + 0.074*"attendance" + 0.067*"master" + 0.059*"researcher" + 0.009*"devastate" + 0.008*"thesis" + 0.006*"willpower" + 0.005*"collapse" + 0.005*"envy"')




0
My thesis is garbage and I won't be able to defend it. I have a bachelors in psychology. I need a masters to do what I want to do (UX researcher). I just uncovered some mistakes in my code for my thesis study... like serious mistakes. Like what the fuck was I doing mistakes. And in my text files of àll places.

My advisor said they'd help me but I know we can't fix this. I just wanted that degree so badly so that I could finally collapse for a while without worrying about my future being over.

Now how am I supposed to go on after sinking 6 years into a worthless degree while the rest of my life is shit. It's hard not to give in to despair.




1
Is it ok to ask a professor if you can help/work with them on their research if you've never taken a class with them? I'm transferring to a 

In [ ]:
df["txt_raw"][0]

"Racism Making Me Depressed People probably think I weird for being depressed cause of this.  But, first, I think of Youkrane.  Evil country is invading Youkrane because Youkrane good country that is anti-racism and pro-diversity and pro-human-right.   I depressed because of the suffering of Youkrane.  I ask President of America: please don't stop until Youkrane remove evil invader and make all of Youkrane completely free again.  I will try to stay alive if President don't quit removing evil invader from Youkrane.  I think American President should declare Martial Law and temporarily halt all elections until Youkrane is free and evil invaders are defeated. If evil invaders of Youkrane win, then the Earth will be completely defeated, and then nothing left.  Must win at all cost.\r\n\r\nOkay, so I live in America.  And here, so much racism.  why we have border?  all human is human, right?  so, border exist to oppress people.  why not any human allow to come?  why only one people in count